*****
## Step 1: collect billboard data
Songs date as far back as 1958, where I found the first available chart for Hot 100.

In [ ]:
# pip install billboard.py

# in this part we want to build up a data frame of all available charts on billboard
import billboard
import pandas as pd

Hot_10 = pd.DataFrame(columns = ['date','title','artist','rank','weeks','change_in_rank','spotifyID'
    ])


chart = billboard.ChartData('hot-100',date = '2010-12-25') # this breaks sometimes so have to rerun and change date
date = chart.date
prev_year,prev_month = date.split('-')[0],date.split('-')[1]

while chart.previousDate:
    year,month = date.split('-')[0],date.split('-')[1]
    if prev_month != month:
        print 'chart year:{},month:{}'.format(year,month)
    for i in range(10):
        song = chart[i]
        Hot_10 = Hot_10.append([(date,song.title,song.artist,song.rank,song.weeks,song.change,song.spotifyID)])
    chart = billboard.ChartData('hot-100',chart.previousDate)
    prev_year,prev_month = year,month
    date = chart.date
    no += 1


In [ ]:
# the scrapping breaks sometimes due to website control so have to rerun and change date
chart = billboard.ChartData('hot-100',date = '1958-08-09') 
date = chart.date
prev_year,prev_month = date.split('-')[0],date.split('-')[1]

while chart.previousDate:
    year,month = date.split('-')[0],date.split('-')[1]
    if prev_month != month:
        print 'chart year:{},month:{}'.format(year,month)
    for i in range(10):
        song = chart[i]
        Hot_10 = Hot_10.append([(date,song.title,song.artist,song.rank,song.weeks,song.change,song.spotifyID)])
    chart = billboard.ChartData('hot-100',chart.previousDate)
    prev_year,prev_month = year,month
    date = chart.date
    no += 1

In [ ]:
Hot_10_dropped = Hot_10.drop(Hot_10.columns.values[7:],axis=1)
Hot_10_dropped.columns = ['date','title','artist','rank','weeks','change_in_rank','spotifyID']
Hot_10_dropped.to_pickle('Billboard_data')

In [ ]:
import pandas as pd
top10 = pd.read_pickle('Billboard_data')
top10.head()